In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
import json
from tqdm import tqdm
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
from IPython.core.display import HTML
from IPython.core import display

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
HTML("""

<style>
mark{
 background-color:#c0ffc8;
}
</style>

""")


In [ ]:

def text_cleaning(text):
    '''
    Converts all text to lower case, Removes special charecters, emojis and multiple spaces
    text - Sentence that needs to be cleaned
    '''
#     text = ''.join([k for k in text if k not in string.punctuation])
    text = re.sub('[^A-Za-z0-9]+', ' ', text)
#     text = re.sub("/'+/g", ' ', text)
    
    return text

In [ ]:
train = pd.read_csv('../input/coleridgeinitiative-show-us-the-data/train.csv')
sample_sub = pd.read_csv('../input/coleridgeinitiative-show-us-the-data/sample_submission.csv')
train_files_path = '../input/coleridgeinitiative-show-us-the-data/train'

In [ ]:
def read_append_return(filename, train_files_path=train_files_path, output=''):
    """
    Function to read json file and then return the text data from them and append to the dataframe
    """
    json_path = os.path.join(train_files_path, (filename+'.json'))
    headings = []
    contents = []
    combined = []
    with open(json_path, 'r') as f:
        json_decode = json.load(f)
        for data in json_decode:
            headings.append(data.get('section_title'))
            contents.append(data.get('text'))
            combined.append(data.get('section_title'))
            combined.append(' \\n ')
            combined.append(data.get('text'))
    
    all_headings = ' '.join(headings)
    all_contents = ' '.join(contents)
    all_data = '. '.join(combined)
    
    if output == 'text':
        return all_contents#text_cleaning(all_contents.lower())
    elif output == 'head':
        return all_headings
    else:
        return all_data

In [ ]:
%%time

import joblib

def load_data(df, func=read_append_return):
    pool = joblib.Parallel(4)
    mapper = joblib.delayed(func)
    tasks = [mapper(filename) for filename in df['Id']]
    res = pool(tqdm(tasks))
    return res


train['text'] = load_data(train)

In [ ]:
train.head()

In [ ]:
idx = np.random.randint(0, len(train), 500)

In [ ]:
def show_label(row, is_lower=False):
    title = row.pub_title
    txt = row.text
    label = row.dataset_label
    clean_label = row.cleaned_label
    if is_lower:
        txt = text_cleaning(txt)
        start_idx = txt.lower().index(clean_label)
    else:
        start_idx = txt.index(label)
    end_idx = start_idx + len(label)
    context_size = 100

    p1 = txt[max(0, start_idx-context_size):start_idx]
    p2 = txt[end_idx: min(end_idx+context_size, len(txt))]
    seg = f'<h3>{title}</h3><p>{p1}<mark>{label}</mark>{p2}</p>'
    return HTML(seg)

In [ ]:
for i in idx:
    row = train.iloc[i]
    print(i)
    try:
        a = show_label(row)
    except:
        print('find failed', i)
        a = show_label(row, True)
        
    display.display(a)
    print()